# MILESTONE 2
## REZA AMINI

* **Importing Libraries**

In [52]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [53]:
def price_simulator_multiple_trajectories(
                    random_seed:int,
                    num_asset:int,
                    init_price:np.ndarray,
                    interest_rate:np.ndarray,
                    sigma:np.ndarray,
                    num_stages:int,
                    delta_t:float,
                    num_trajectories:int):
    
    """
    *** GENERATING 3D PRICE MATRIX (J * T * M) ***
    
    random_seed: seed for random generator
    num_asset: number of assets (J)
    init_price: initial prices of assets
    interest_rate: interest rate of each asset
    sigma: stock price volatility
    num_stages: number of permissible exercise stages (T)
    delta_t: time difference between two subsequent stages
    num_trajectories: number of trajectories for each asset (M)
    
    """

    assert len(init_price) == num_asset
    assert len(interest_rate) == num_asset
    assert len(sigma) == num_asset

    np.random.seed(random_seed)
    prices = np.zeros([num_asset, num_stages, num_trajectories])
    for i in range(num_asset):
        prices[i, 0, :]    = init_price[i]*np.ones([num_trajectories])
    
    for i in range(1, int(num_stages)):
        for j in range(num_asset):
            drift           = (interest_rate[j] - 0.5 * sigma[j]**2) * delta_t
            Z               = np.random.normal(0., 1., num_trajectories)
            diffusion       = Z * (np.sqrt(delta_t)) * sigma[j]
            prices[j, i, :]    = prices[j, (i - 1), :]*np.exp(drift + diffusion)
    return prices

In [54]:
def plot_prices_multiple_trajectories(
                    random_seed:int,
                    prices:np.ndarray,
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int):
    
    """
    *** PLOTTING 3D PRICE MATRIX (J * T * M) ***
    
    random_seed: seed for random generator
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    
    """
        
    random.seed(random_seed)
    
    fig, ax  = plt.subplots(figsize=(6, 4),dpi=100)
    cols = ["g", "b", "r", "y", "k", "c", "m"]
    asset_color = random.sample(cols, num_asset)
    for i in range(num_asset):
        for j in range(num_trajectories):
            ax.plot(prices[i, :, j], lw = 2, alpha = .5, 
                    label = "Asset " + str(i + 1), marker = '.', color = asset_color[i])

    min_p,max_p = float(np.min(prices)), float(np.max(prices))
    ax.set_xticks(np.arange(0, num_stages + 1, 4))
    ax.set_yticks(np.round(np.arange(min_p, max_p + 1, int((max_p - min_p)/10.))))
    ax.set_xlabel(r'stage ($t$)')
    ax.set_ylabel(r'price ($p_{t,j}$)')
    plt.legend()
    plt.show()
    return True

In [55]:
def knock_out_indicator_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray, 
                    barrier_price:float):
    
    """
    *** GENERATING KNOCk-OUT INDICATOR MATRIX (M * T) ***
    
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    barrier_price: the barrier knock-out price value
    
    """
    
    y = np.zeros([num_trajectories, num_stages], dtype = np.int8)
    
    for i in range(num_trajectories):
        for j in range(num_stages):
            if j == 0:
                if np.max(prices[:, 0, i]) >= barrier_price:
                    y[i, 0] = 1
            else:
                if np.max(prices[:, j, i]) >= barrier_price or y[i, (j - 1)] == 1:
                    y[i, j] = 1
                    
    return y

In [56]:
def payoff_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    knock_out:np.ndarray,
                    strike_price:float):
    
    """
    *** GENERATING PAYOFF MATRIX (M * T) ***
    
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    knock_out: knock-out indicator matrix
    strike_price: the target strike price
    
    """
    
    g = np.zeros([num_trajectories, num_stages])
    
    for i in range(num_trajectories):
        for j in range(num_stages):
            g[i, j] = np.max([np.max(prices[:, j, i]) - strike_price, 0.])*float(1 - knock_out[i, j])
            
    return g

In [57]:
def feature_matrix_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    knock_out:np.ndarray,
                    payoff:np.ndarray,
                    method:str,
                    centroid:float = 0.,
                    rho:float = 0.5):
    
    """
    *** GENERATING FEATURE MATRIX (M * J * T) ***

    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    knock_out: knock-out indicator matrix
    payoff: payoff matrix
    method: continuation function approximation method
    centroid: radial basis function centroid for kernel method
    rho: bandwidth length for kernel method
    
    """
    
    if method == "linear":
        
        F = np.zeros([num_trajectories, num_asset, num_stages]) 
        for i in range(num_trajectories):
            for j in range(num_asset):
                F[i, j, :] = prices[j, :, i]*(1 - knock_out[i, :])
    if method == "DFM":
        
        F = np.zeros([num_trajectories, num_asset + 2, num_stages])
        for i in range(num_trajectories):
            for j in range(num_asset + 2):
                if j == 0:
                    F[i, j, :] = np.ones([num_stages])*(1 - knock_out[i, :])
                elif j == 1:
                    F[i, j, :] = payoff[i, :]*(1 - knock_out[i, :])
                else:
                    F[i, j, :] = prices[j - 2, :, i]*(1 - knock_out[i, :])
    
    if method == "kernel":
        F = np.zeros([num_trajectories, num_asset, num_stages])
        for i in range(num_trajectories):
            for j in range(num_asset):
                for k in range(num_stages):
                    F[i, j, k] = np.exp(-np.linalg.norm(prices[:, k, i] - centroid, 2)**2/2./rho)*(1 - knock_out[i, k])
                
    return F

In [58]:
def fit_CFA(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    feature_matrix:np.ndarray,
                    payoff:np.ndarray,
                    gamma:float):
    
    """
    *** LSM TO FIT CFA ***

    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    feature_matrix: feature matrix
    payoff: payoff matrix
    gamma: discount factor
    
    """
    
    B = feature_matrix.shape[1]
    w = np.zeros([B, num_stages])
    w[:, (num_stages - 1)] = 0.
    gamma = np.exp(-delta_t*np.mean(interest_rate))
    for i in range(num_stages - 2, -1, -1):
        ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
        w[:, i], _, _, _ = np.linalg.lstsq(feature_matrix[:, :, i], ct, rcond = None)
    
    return w

In [59]:
def policy_simulation(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    weight:np.ndarray,
                    payoff:np.ndarray,
                    knock_out:np.ndarray,
                    feature_matrix:np.ndarray,
                    gamma:float):
    
    """
    *** SIMULATE POLICY AND RETURN AVERAGE PAYOFF ***

    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    weight: weights fitted to CFA
    knock_out: knock-out indicator matrix
    payoff: payoff matrix
    feature_matrix: feature matrix
    gamma: discount factor
    
    """
    
    c = np.zeros([num_trajectories, num_stages])
    
    r = np.ones([num_trajectories])*np.NaN
    for i in range(num_trajectories):
        for j in range(num_stages - 1):
            
            c[i, j] = gamma*np.max([payoff[i, j + 1], (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j])])
            
            if c[i, j] < payoff[i, j]:
                r[i] = gamma**(j + 1)*payoff[i, j]
                
        if np.isnan(r[i]):
            r[i] = gamma**num_stages*payoff[i, (num_stages - 1)]
            
    return np.sum(r)/num_trajectories

# Q1 - Linear CFA

In [60]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")



yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

********************************
Optimal Weights:
[[0.11902592 0.11176504 0.11570527 0.11620075 0.11543369 0.11666073
  0.11739822 0.11726307 0.11879114 0.11967547 0.12082185 0.11985454
  0.11809815 0.11799482 0.11754691 0.12097214 0.11837185 0.122172
  0.12335963 0.12290818 0.12213626 0.11543816 0.11384018 0.11598679
  0.11327868 0.11474525 0.1077468  0.10626145 0.09580408 0.10435305
  0.10966678 0.1219579  0.12959532 0.13623952 0.13198092 0.11313076
  0.11415177 0.12253142 0.12352511 0.12901487 0.11203128 0.12560812
  0.12503752 0.12602124 0.10253761 0.10794095 0.11059758 0.1105598
  0.10834199 0.09959389 0.08578618 0.06848125 0.06411856 0.        ]
 [0.11902592 0.12239996 0.12154787 0.12108185 0.12465504 0.12718571
  0.12602822 0.12505166 0.1212776  0.12059869 0.1230706  0.1216749
  0.12044727 0.11939519 0.11756193 0.12015354 0.11627054 0.11565205
  0.11930943 0.11480777 0.11037857 0.10786673 0.11301675 0.10667631
  0.10712978 0.11206146 0.11039437 0.1187838  0.12170415 0.10968019
 

# Q2 - DFM CFA

In [61]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)


yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")



yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by DFM method is larger than linear CFA.")

********************************
Optimal Weights:
[[ 1.09891077e-03 -1.23514944e+01 -9.78014639e+00 -7.71190728e+00
  -6.69498878e+00 -4.11345158e+00 -2.46879182e+00 -4.87541243e-01
   1.84776967e-01  2.42554698e+00  3.95807920e+00  6.55606921e+00
   7.86290916e+00  9.57210567e+00  1.01885936e+01  1.20713805e+01
   1.13105249e+01  1.30616875e+01  1.11046536e+01  1.07782210e+01
   1.21180061e+01  1.36794033e+01  1.48471513e+01  1.75269317e+01
   1.82978280e+01  2.02375396e+01  1.74708277e+01  1.70219958e+01
   1.64327947e+01  1.70741371e+01  1.61547328e+01  1.57834091e+01
   1.56111622e+01  1.35711768e+01  1.21548153e+01  1.34354868e+01
   1.19083601e+01  1.06102183e+01  4.02041679e+00  3.98576005e+00
   2.59624094e+00  4.63601421e+00  4.02122605e+00  2.68550722e+00
   3.49773838e+00  4.22344599e+00  4.49197893e+00  4.31511130e+00
   2.89028601e+00 -9.98706591e-03 -1.79494547e+00 -4.81266549e+00
  -4.64355406e+00  0.00000000e+00]
 [ 0.00000000e+00  2.07982839e-01  9.74217338e-02  1.1179

# Q3 - Kernel CFA

### Kernel CFA - 25 Percentile Centroid and Bandwidth Equals to 10000

In [62]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000. # bandwidth for kernel method
centroid         = np.percentile(prices, 25) # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by kernel method with 1st quartile as centroid and bandwidth equals to 10000 is smaller than both CFA and linear method.")

********************************
Optimal Weights:
[[14.53721241 14.64531218 14.75274779 14.86563575 14.97691377 15.08908473
  15.20374825 15.322356   15.43670997 15.54516149 15.66518972 15.78700805
  15.90513039 16.01905317 16.13767432 16.24566507 16.34888926 16.43710455
  16.52745703 16.59503172 16.67384614 16.74358286 16.78872501 16.86364454
  16.91725786 16.96732715 16.96757587 17.01394383 16.99577624 17.03379987
  17.03248301 17.0175307  16.98535138 16.9485271  16.88361533 16.84486066
  16.77229544 16.65847502 16.55483022 16.41213335 16.23031884 16.08398614
  15.86117992 15.65412012 15.32915976 15.00566786 14.61653491 14.04953515
  13.41115981 12.58093555 11.48480035  9.90907015  7.40295986  0.        ]
 [14.53721241 14.64531218 14.75274779 14.86563575 14.97691377 15.08908473
  15.20374825 15.322356   15.43670997 15.54516149 15.66518972 15.78700805
  15.90513039 16.01905317 16.13767432 16.24566507 16.34888926 16.43710455
  16.52745703 16.59503172 16.67384614 16.74358286 16.78872501

### Kernel CFA - 50 Percentile (Median) Centroid and Bandwidth Equals to 10000

In [63]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000. # bandwidth for kernel method
centroid         = np.percentile(prices, 50) # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")



yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by kernel method with 2nd quartile (median) as centroid and bandwidth equals to 10000 is again smaller than both CFA and linear method and larger than 1st quartile centroid.")

********************************
Optimal Weights:
[[13.74324544 13.82685315 13.91374346 14.00340788 14.09423832 14.1846006
  14.27770636 14.37406777 14.46753391 14.55959115 14.65724555 14.75868285
  14.85320197 14.94320533 15.03694357 15.12803182 15.20743532 15.28094949
  15.35190021 15.39940722 15.46795116 15.52145067 15.56071674 15.62832013
  15.67839361 15.72861926 15.73047758 15.77759008 15.75840442 15.79571153
  15.80158672 15.79596736 15.77381125 15.75033725 15.69967747 15.67265918
  15.62341846 15.5255325  15.45169668 15.33463768 15.18777799 15.08019724
  14.89987049 14.74205555 14.4709772  14.20682649 13.88174181 13.39430321
  12.84835736 12.12234981 11.14439057  9.71683567  7.39111478  0.        ]
 [13.74324544 13.82685315 13.91374346 14.00340788 14.09423832 14.1846006
  14.27770636 14.37406777 14.46753391 14.55959115 14.65724555 14.75868285
  14.85320197 14.94320533 15.03694357 15.12803182 15.20743532 15.28094949
  15.35190021 15.39940722 15.46795116 15.52145067 15.56071674 1

### Kernel CFA - 75 Percentile Centroid and Bandwidth Equals to 10000

In [64]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000. # bandwidth for kernel method
centroid         = np.percentile(prices, 75) # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)



print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by kernel method with 3rd quartile as centroid and bandwidth equals to 10000 is smaller than both CFA and linear method and larger than 1st and 2nd quartile centroids.")

********************************
Optimal Weights:
[[13.74486849 13.79995793 13.86385628 13.92720664 13.99576116 14.06131812
  14.13057967 14.20221986 14.27370586 14.34899388 14.42144391 14.50105265
  14.5686427  14.63133825 14.69658138 14.76861297 14.8201103  14.87664256
  14.92503338 14.94627186 15.00113924 15.03345991 15.06206381 15.12148724
  15.16500814 15.21467272 15.21225869 15.26151    15.23694447 15.27163727
  15.28550447 15.28891526 15.2743473  15.26095899 15.21962619 15.19899107
  15.17480281 15.0869912  15.04037861 14.93847381 14.81731785 14.74265139
  14.60002955 14.4878094  14.26615661 14.05715045 13.78918457 13.37154347
  12.91302822 12.27728755 11.3969382  10.08302968  7.86802157  0.        ]
 [13.74486849 13.79995793 13.86385628 13.92720664 13.99576116 14.06131812
  14.13057967 14.20221986 14.27370586 14.34899388 14.42144391 14.50105265
  14.5686427  14.63133825 14.69658138 14.76861297 14.8201103  14.87664256
  14.92503338 14.94627186 15.00113924 15.03345991 15.06206381

### Kernel CFA - Median Centroid and Bandwidth Equals to 1

In [65]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 1. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1. # bandwidth for kernel method
centroid         = np.percentile(prices, 50) # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")



yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by kernel method with 2nd quartile (median) as centroid and bandwidth equals to 1, comparing to bandwidth of 10000, dicreased the payoff.")

********************************
Optimal Weights:
[[5.99457515e+07 1.22465768e+02 1.02965331e+02 1.26420930e+02
  5.07228006e+01 3.17031726e+02 9.43958541e+01 9.38036907e+03
  4.81257807e+05 6.83593141e+01 7.99504666e+05 8.14281597e+02
  9.76261478e+02 1.26936850e+03 4.08949739e+02 2.71930663e+04
  1.35565537e+04 2.05350888e+02 5.74746689e+03 2.87789863e+02
  1.76776573e+02 8.07320252e+13 9.18939544e+03 7.82963423e+05
  6.75927493e+03 5.80046673e+08 7.51791546e+12 9.60262645e+06
  2.97406996e+09 6.63372906e+05 9.47912113e+12 9.01534900e+03
  1.68356639e+06 1.57426075e+04 2.08938493e+09 2.03385604e+05
  1.01817091e+14 1.51460760e+17 8.31288162e+13 5.56112059e+18
  2.21697659e+06 6.99701042e+10 4.12877998e+05 7.11618427e+06
  7.13917176e+13 3.43334709e+16 2.13888195e+12 7.06588310e+02
  2.26471037e+07 6.26001308e+06 1.19832759e+08 1.74729419e+11
  9.83509866e+05 0.00000000e+00]
 [5.99457515e+07 1.22465768e+02 1.02965331e+02 1.26420930e+02
  5.07228006e+01 3.17031726e+02 9.43958541e+01 9.

### Kernel CFA - 99% Percentile Centroid and Very Large Bandwidth

In [66]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000000. # bandwidth for kernel method
centroid         = np.percentile(prices, 99) # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


print("********************************")
print("Optimal Weights:")
print(w)
print("********************************\n\n\n")



yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)
print("\n\n\n")
print("The payoff value obtained by kernel method with 99% percentile as centroid and a very large bandwidth, although close to the largest possible value we can get from kernel method given our criteria, is still smaller than linear and DFM methods.")

********************************
Optimal Weights:
[[12.01031723 12.04372551 12.07722675 12.11082116 12.14450904 12.17829061
  12.21216616 12.24613594 12.28009903 12.3130547  12.34468498 12.38198441
  12.41100909 12.43690585 12.45895721 12.48324463 12.49907695 12.51563707
  12.53277608 12.52539206 12.54875333 12.55570557 12.53743266 12.55709808
  12.57382684 12.59038143 12.56293263 12.58897863 12.54921672 12.56469204
  12.56584008 12.55252557 12.53359972 12.50774665 12.46973217 12.45935923
  12.4396402  12.36899219 12.3364353  12.26278065 12.16260114 12.12921271
  12.02705009 11.96081357 11.8014881  11.66726966 11.47996578 11.1557178
  10.81846854 10.33127034  9.65756661  8.60437357  6.7312649   0.        ]
 [12.01031723 12.04372551 12.07722675 12.11082116 12.14450904 12.17829061
  12.21216616 12.24613594 12.28009903 12.3130547  12.34468498 12.38198441
  12.41100909 12.43690585 12.45895721 12.48324463 12.49907695 12.51563707
  12.53277608 12.52539206 12.54875333 12.55570557 12.53743266 